In [1]:
# import pandas as pd

# # Load the XPT file
# df = pd.read_sas("GHB_L.xpt", format="xport")
# # Save as CSV
# df.to_csv("output.csv", index=False)

# print("Conversion completed successfully!")

In [2]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import os

In [3]:


# Define file paths
file_paths = {
    "DEMO_L": "DEMO_L (1).xpt",
    "DBQ_L": "DBQ_L.xpt",
    "DR1TOT_L": "DR1TOT_L.xpt",
    "DR2TOT_L": "DR2TOT_L.xpt",
    "ALQ_L": "ALQ_L.xpt",
    "SMQ_L": "SMQ_L.xpt",
    "TCHOL_L": "TCHOL_L.xpt",
    "FASTQX_L": "FASTQX_L.xpt",
    "BPXO_L": "BPXO_L.xpt",
    "BMX_L": "BMX_L.xpt",
    "PAQ_L": "PAQ_L (1).xpt"
}

# Load all files into dataframes
dfs = {name: pd.read_sas(path, format="xport") for name, path in file_paths.items()}

# Merge all dataframes on 'SEQN' (Participant ID)
merged_df = dfs["DEMO_L"]  # Start with the demographics dataset

for name, df in dfs.items():
    if name != "DEMO_L":
        merged_df = pd.merge(merged_df, df, on="SEQN", how="left")  # Left join to retain all participants

# Display final merged dataframe shape
print("Merged DataFrame Shape:", merged_df.shape)

Merged DataFrame Shape: (11933, 380)


In [4]:
merged_df

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,BMIWAIST,BMXHIP,BMIHIP,PAD790Q,PAD790U,PAD800,PAD810Q,PAD810U,PAD820,PAD680
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,102.9,NaN,3.000000e+00,b'W',45.0,3.000000e+00,b'W',45.0,360.0
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,112.4,NaN,4.000000e+00,b'W',45.0,3.000000e+00,b'W',45.0,480.0
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,98.0,NaN,1.000000e+00,b'W',20.0,5.397605e-79,b'',NaN,240.0
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11928,142306.0,12.0,2.0,1.0,9.0,NaN,2.0,2.0,1.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11929,142307.0,12.0,2.0,2.0,49.0,NaN,4.0,4.0,2.0,NaN,...,1.0,NaN,1.0,3.000000e+00,b'W',15.0,5.397605e-79,b'',NaN,480.0
11930,142308.0,12.0,2.0,1.0,50.0,NaN,2.0,2.0,1.0,NaN,...,NaN,97.7,NaN,1.000000e+00,b'W',45.0,5.397605e-79,b'',NaN,600.0
11931,142309.0,12.0,2.0,1.0,40.0,NaN,2.0,2.0,1.0,NaN,...,NaN,103.3,NaN,2.000000e+00,b'D',15.0,5.397605e-79,b'',NaN,240.0


In [5]:
merged_df.isnull().sum()

SEQN           0
SDDSRVYR       0
RIDSTATR       0
RIAGENDR       0
RIDAGEYR       0
            ... 
PAD800      5543
PAD810Q     3794
PAD810U     3780
PAD820      8246
PAD680      3795
Length: 380, dtype: int64

In [6]:
# Adjust feature selection based on available columns
adjusted_features = [
    "RIDAGEYR",  # Age (Check if present)
    "BMXBMI",  # Body Mass Index
    "BMXWAIST",  # Waist circumference
    "BPXOSY1",  # Systolic blood pressure (Renamed from BPXSY1)
    "BPXODI1",  # Diastolic blood pressure (Renamed from BPXDI1)
    "LBXTC",  # Total cholesterol
    "SMQ020",  # Smoking status (Yes/No)
    "ALQ130",  # Alcohol intake (drinks per week)
    "DR1TKCAL",  # Total calorie intake (Day 1)
    "DR2TKCAL",  # Total calorie intake (Day 2)
]

# Keep only the adjusted features
df_selected = merged_df[adjusted_features]

# Handle missing values: Impute numerical columns with median
imputer = SimpleImputer(strategy="median")
df_imputed = pd.DataFrame(imputer.fit_transform(df_selected), columns=adjusted_features)

# Normalize numerical features
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=adjusted_features)

# One-hot encode categorical variables (if any exist in the selected features)
encoder = OneHotEncoder(drop="first", sparse_output=False)
categorical_vars = ["SMQ020"]  # Smoking status (binary Yes/No)

df_encoded = pd.DataFrame(encoder.fit_transform(df_imputed[categorical_vars]), columns=encoder.get_feature_names_out(categorical_vars))

# Combine scaled numerical and encoded categorical data
df_final = pd.concat([df_scaled.drop(columns=categorical_vars), df_encoded], axis=1)

# Show final processed dataset summary
df_final.describe()


,RIDAGEYR,BMXBMI,BMXWAIST,BPXOSY1,BPXODI1,LBXTC,ALQ130,DR1TKCAL,DR2TKCAL,SMQ020_2.0,SMQ020_7.0,SMQ020_9.0
count,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,11933.000000,11933.000000,11933.000000
mean,2.381774e-17,1.357611e-16,-1.121816e-15,-1.119434e-16,-1.476700e-16,-3.221349e-16,-1.637470e-17,2.858129e-17,-1.524335e-16,0.727059,0.000587,0.000587
std,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,0.445489,0.024214,0.024214
min,-1.496738e+00,-2.315642e+00,-2.873917e+00,-3.888546e+00,-4.177909e+00,-3.666021e+00,-7.183549e-02,-2.779571e+00,-2.995023e+00,0.000000,0.000000,0.000000
25%,-9.889434e-01,-4.807240e-01,-3.887813e-01,-4.360702e-01,-3.674176e-01,-2.808908e-01,-4.074325e-02,-2.609641e-01,-1.021501e-01,0.000000,0.000000,0.000000
50%,-5.147699e-02,-8.752717e-02,2.175877e-02,-9.759218e-02,-4.987662e-02,-6.349704e-02,-4.074325e-02,-1.132496e-01,-1.021501e-01,1.000000,0.000000,0.000000
75%,9.250505e-01,3.202325e-01,4.213511e-01,1.731902e-01,2.676644e-01,1.538967e-01,-4.074325e-02,4.938559e-02,-1.021501e-01,1.000000,0.000000,0.000000
max,1.628150e+00,6.960889e+00,5.183616e+00,7.687402e+00,7.359413e+00,8.011126e+00,3.095821e+01,1.280655e+01,1.346925e+01,1.000000,1.000000,1.000000


In [7]:
# List all available column names in the merged dataset
available_columns = merged_df.columns.tolist()
available_columns

['SEQN',
 'SDDSRVYR',
 'RIDSTATR',
 'RIAGENDR',
 'RIDAGEYR',
 'RIDAGEMN',
 'RIDRETH1',
 'RIDRETH3',
 'RIDEXMON',
 'RIDEXAGM',
 'DMQMILIZ',
 'DMDBORN4',
 'DMDYRUSR',
 'DMDEDUC2',
 'DMDMARTZ',
 'RIDEXPRG',
 'DMDHHSIZ',
 'DMDHRGND',
 'DMDHRAGZ',
 'DMDHREDZ',
 'DMDHRMAZ',
 'DMDHSEDZ',
 'WTINT2YR',
 'WTMEC2YR',
 'SDMVSTRA',
 'SDMVPSU',
 'INDFMPIR',
 'DBQ010',
 'DBD030',
 'DBD041',
 'DBD050',
 'DBD055',
 'DBD061',
 'DBQ073A',
 'DBQ073B',
 'DBQ073C',
 'DBQ073D',
 'DBQ073E',
 'DBQ073U',
 'DBQ301',
 'DBQ330',
 'DBQ360',
 'DBQ370',
 'DBD381',
 'DBQ390',
 'DBQ400',
 'DBD411',
 'DBQ421',
 'DBQ424',
 'DBQ930',
 'DBQ935',
 'DBQ940',
 'DBQ945',
 'WTDRD1_x',
 'WTDR2D_x',
 'DR1DRSTZ',
 'DR1EXMER',
 'DRABF_x',
 'DRDINT_x',
 'DR1DBIH',
 'DR1DAY',
 'DR1LANG',
 'DR1MRESP',
 'DR1HELP',
 'DBQ095Z',
 'DBD100',
 'DRQSPREP',
 'DR1STY',
 'DR1SKY',
 'DRQSDIET',
 'DRQSDT1',
 'DRQSDT2',
 'DRQSDT3',
 'DRQSDT4',
 'DRQSDT5',
 'DRQSDT6',
 'DRQSDT7',
 'DRQSDT8',
 'DRQSDT9',
 'DRQSDT10',
 'DRQSDT11',
 'DRQSDT12',
 'DRQSD

In [ ]:
# # Load DEMO_L separately to check for "RIDAGEYR" (Age)
# import pandas as pd
# from pyreadstat import read_xport

# # Reload DEMO_L file to inspect its columns
# # demo_file_path = "DEMO_L (1).xpt"

# demo_df = pd.read_sas(r"C:\Users\Vernon\Desktop\MyCodingWorkSpace\MachineLearning\Finals_Adv_ML\DEMO_L (1).xpt", format="xport", encoding="latin1")



# # Check if "RIDAGEYR" exists in DEMO_L
# demo_df.columns.tolist()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Vernon\\Desktop\\MyCodingWorkSpace\\MachineLearning\\Finals_Adv_ML\\DEMO_L (1).xpt'

In [ ]:
# # Re-merge DEMO_L with the existing merged dataset using SEQN as the key
# merged_df = pd.merge(merged_df, demo_df[["SEQN", "RIDAGEYR"]], on="SEQN", how="left")

# # Verify that RIDAGEYR is now included
# "RIDAGEYR" in merged_df.columns


NameError: name 'demo_df' is not defined

In [ ]:
# # Attempt to load the DEMO_L file using pandas' built-in SAS reader
# demo_df = pd.read_sas("DEMO_L (1).xpt", format="xport")

# # List columns in DEMO_L to check for "RIDAGEYR"
# demo_df.columns.tolist()


['SEQN',
 'SDDSRVYR',
 'RIDSTATR',
 'RIAGENDR',
 'RIDAGEYR',
 'RIDAGEMN',
 'RIDRETH1',
 'RIDRETH3',
 'RIDEXMON',
 'RIDEXAGM',
 'DMQMILIZ',
 'DMDBORN4',
 'DMDYRUSR',
 'DMDEDUC2',
 'DMDMARTZ',
 'RIDEXPRG',
 'DMDHHSIZ',
 'DMDHRGND',
 'DMDHRAGZ',
 'DMDHREDZ',
 'DMDHRMAZ',
 'DMDHSEDZ',
 'WTINT2YR',
 'WTMEC2YR',
 'SDMVSTRA',
 'SDMVPSU',
 'INDFMPIR']

In [ ]:
# # Re-merge DEMO_L with the existing merged dataset using SEQN as the key
# merged_df = pd.merge(merged_df, demo_df[["SEQN", "RIDAGEYR"]], on="SEQN", how="left")

# # Verify that RIDAGEYR is now included
# "RIDAGEYR" in merged_df.columns


True

In [10]:
# Adjust feature selection again now that RIDAGEYR is available



adjusted_features = [
    "RIDAGEYR",  # Age
    "BMXBMI",  # Body Mass Index
    "BMXWAIST",  # Waist circumference
    "BPXOSY1",  # Systolic blood pressure
    "BPXODI1",  # Diastolic blood pressure
    "LBXTC",  # Total cholesterol
    "SMQ020",  # Smoking status (Yes/No)
    "ALQ130",  # Alcohol intake (drinks per week)
    "DR1TKCAL",  # Total calorie intake (Day 1)
    "DR2TKCAL",  # Total calorie intake (Day 2)
]

# Keep only the adjusted features
df_selected = merged_df[adjusted_features]

# Handle missing values: Impute numerical columns with median
imputer = SimpleImputer(strategy="median")
df_imputed = pd.DataFrame(imputer.fit_transform(df_selected), columns=adjusted_features)

# Normalize numerical features
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=adjusted_features)

# One-hot encode categorical variables (if any exist in the selected features)
encoder = OneHotEncoder(drop="first", sparse_output=False)
categorical_vars = ["SMQ020"]  # Smoking status (binary Yes/No)

df_encoded = pd.DataFrame(encoder.fit_transform(df_imputed[categorical_vars]), columns=encoder.get_feature_names_out(categorical_vars))

# Combine scaled numerical and encoded categorical data
df_final = pd.concat([df_scaled.drop(columns=categorical_vars), df_encoded], axis=1)

# Show final processed dataset summary
df_final.describe()

,RIDAGEYR,BMXBMI,BMXWAIST,BPXOSY1,BPXODI1,LBXTC,ALQ130,DR1TKCAL,DR2TKCAL,SMQ020_2.0,SMQ020_7.0,SMQ020_9.0
count,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,11933.000000,11933.000000,11933.000000
mean,2.381774e-17,1.357611e-16,-1.121816e-15,-1.119434e-16,-1.476700e-16,-3.221349e-16,-1.637470e-17,2.858129e-17,-1.524335e-16,0.727059,0.000587,0.000587
std,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,0.445489,0.024214,0.024214
min,-1.496738e+00,-2.315642e+00,-2.873917e+00,-3.888546e+00,-4.177909e+00,-3.666021e+00,-7.183549e-02,-2.779571e+00,-2.995023e+00,0.000000,0.000000,0.000000
25%,-9.889434e-01,-4.807240e-01,-3.887813e-01,-4.360702e-01,-3.674176e-01,-2.808908e-01,-4.074325e-02,-2.609641e-01,-1.021501e-01,0.000000,0.000000,0.000000
50%,-5.147699e-02,-8.752717e-02,2.175877e-02,-9.759218e-02,-4.987662e-02,-6.349704e-02,-4.074325e-02,-1.132496e-01,-1.021501e-01,1.000000,0.000000,0.000000
75%,9.250505e-01,3.202325e-01,4.213511e-01,1.731902e-01,2.676644e-01,1.538967e-01,-4.074325e-02,4.938559e-02,-1.021501e-01,1.000000,0.000000,0.000000
max,1.628150e+00,6.960889e+00,5.183616e+00,7.687402e+00,7.359413e+00,8.011126e+00,3.095821e+01,1.280655e+01,1.346925e+01,1.000000,1.000000,1.000000


In [ ]:
df_final.head()

,RIDAGEYR,BMXBMI,BMXWAIST,BPXOSY1,BPXODI1,LBXTC,ALQ130,DR1TKCAL,DR2TKCAL,SMQ020_2.0,SMQ020_7.0,SMQ020_9.0
0,0.182890,-0.000150,0.328295,1.120929,2.702145,2.607340,-0.040743,-0.183377,-1.321726,0.0,0.0,0.0
1,1.081295,0.946435,1.226010,0.173190,1.220287,1.054528,-0.009651,1.310181,0.589694,0.0,0.0,0.0
2,0.221951,0.393047,0.065550,-0.503766,0.691052,0.216009,-0.071835,0.197100,-0.621837,1.0,0.0,0.0
3,-1.301432,-0.466161,-1.198914,-0.097592,-0.049877,-0.063497,-0.040743,-0.278869,-1.276676,1.0,0.0,0.0
4,-1.418615,-0.087527,0.021759,-0.097592,-0.049877,-0.063497,-0.040743,-0.956267,-1.278285,1.0,0.0,0.0


In [11]:
# Fix OneHotEncoder issue by using sparse=False
encoder = OneHotEncoder(drop="first", sparse_output=False)

categorical_vars = ["SMQ020"]  # Smoking status (binary Yes/No)

# Encode categorical variables
df_encoded = pd.DataFrame(encoder.fit_transform(df_imputed[categorical_vars]), columns=encoder.get_feature_names_out(categorical_vars))

# Combine scaled numerical and encoded categorical data
df_final = pd.concat([df_scaled.drop(columns=categorical_vars), df_encoded], axis=1)

# Show final processed dataset summary
df_final.describe()


,RIDAGEYR,BMXBMI,BMXWAIST,BPXOSY1,BPXODI1,LBXTC,ALQ130,DR1TKCAL,DR2TKCAL,SMQ020_2.0,SMQ020_7.0,SMQ020_9.0
count,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,1.193300e+04,11933.000000,11933.000000,11933.000000
mean,2.381774e-17,1.357611e-16,-1.121816e-15,-1.119434e-16,-1.476700e-16,-3.221349e-16,-1.637470e-17,2.858129e-17,-1.524335e-16,0.727059,0.000587,0.000587
std,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,1.000042e+00,0.445489,0.024214,0.024214
min,-1.496738e+00,-2.315642e+00,-2.873917e+00,-3.888546e+00,-4.177909e+00,-3.666021e+00,-7.183549e-02,-2.779571e+00,-2.995023e+00,0.000000,0.000000,0.000000
25%,-9.889434e-01,-4.807240e-01,-3.887813e-01,-4.360702e-01,-3.674176e-01,-2.808908e-01,-4.074325e-02,-2.609641e-01,-1.021501e-01,0.000000,0.000000,0.000000
50%,-5.147699e-02,-8.752717e-02,2.175877e-02,-9.759218e-02,-4.987662e-02,-6.349704e-02,-4.074325e-02,-1.132496e-01,-1.021501e-01,1.000000,0.000000,0.000000
75%,9.250505e-01,3.202325e-01,4.213511e-01,1.731902e-01,2.676644e-01,1.538967e-01,-4.074325e-02,4.938559e-02,-1.021501e-01,1.000000,0.000000,0.000000
max,1.628150e+00,6.960889e+00,5.183616e+00,7.687402e+00,7.359413e+00,8.011126e+00,3.095821e+01,1.280655e+01,1.346925e+01,1.000000,1.000000,1.000000
